In [27]:
import pandas as pd
import numpy as np
import warnings

from sklearn.metrics import precision_score
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier

from sys import platform

if platform == "win32":
    path = 'C:/Users/olive/GitHub/f1-analytics/'
elif platform == "darwin":
    path = '~/Documents/GitHub/f1-analytics/'

warnings.filterwarnings("ignore", category=RuntimeWarning) 
pd.options.mode.chained_assignment = None  # default='warn'

%matplotlib inline

In [28]:
data = pd.read_csv(path+'data/ml_input.csv')
merged = pd.read_csv(path+'data/merged.csv')
parameters = pd.read_csv(path+'parameters/neutralnetworkclassifier.csv')

In [29]:
N = 2021

In [30]:
df = data.copy()
df.podium = df.podium.map(lambda x: 1 if x == 1 else 0)

train = df[df.season < N]
X_train = train.drop(['season', 'round', 'podium', 'driver_points_from_race', 'constructor_points_from_race'], axis=1)
y_train = train.podium

scaler = StandardScaler()
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns = X_train.columns)

In [31]:
data.columns

Index(['season', 'round', 'podium', 'driver_points', 'driver_wins',
       'driver_standings_pos', 'constructor_points', 'constructor_wins',
       'constructor_standings_pos', 'starting_grid', 'driver_points_from_race',
       'constructor_points_from_race', 'stage_q1', 'stage_q2', 'stage_q3',
       'circuit_id_albert_park', 'circuit_id_americas', 'circuit_id_bahrain',
       'circuit_id_baku', 'circuit_id_catalunya', 'circuit_id_hockenheimring',
       'circuit_id_hungaroring', 'circuit_id_imola', 'circuit_id_interlagos',
       'circuit_id_istanbul', 'circuit_id_jeddah', 'circuit_id_losail',
       'circuit_id_marina_bay', 'circuit_id_miami', 'circuit_id_monaco',
       'circuit_id_monza', 'circuit_id_mugello', 'circuit_id_nurburgring',
       'circuit_id_portimao', 'circuit_id_red_bull_ring', 'circuit_id_ricard',
       'circuit_id_rodriguez', 'circuit_id_sepang', 'circuit_id_shanghai',
       'circuit_id_silverstone', 'circuit_id_sochi', 'circuit_id_spa',
       'circuit_id_suzuk

In [32]:
def score_classification(model):
    score = 0
    for circuit in df[df.season == N]['round'].unique():

        test = df[(df.season == N) & (df['round'] == circuit)]
        X_test = test.drop(['season', 'round', 'podium', 'driver_points_from_race', 'constructor_points_from_race'], axis=1)
        y_test = test.podium
        
        #scaling
        X_test = pd.DataFrame(scaler.transform(X_test), columns = X_test.columns)

        # make predictions
        prediction_df = pd.DataFrame(model.predict_proba(X_test), columns = ['proba_0', 'proba_1'])
        prediction_df['actual'] = y_test.reset_index(drop=True)
        prediction_df.sort_values('proba_1', ascending = False, inplace = True)
        prediction_df.reset_index(inplace=True, drop=True)
        prediction_df['predicted'] = prediction_df.index
        prediction_df['predicted'] = prediction_df.predicted.map(lambda x: 1 if x == 0 else 0)

        score += precision_score(prediction_df.actual, prediction_df.predicted)

    model_score = score / df[df.season == N]['round'].unique().max()
    return model_score

In [33]:
def get_predictions(model):
    predictions = []
    for circuit in df[df.season == N]['round'].unique():

        test = df[(df.season == N) & (df['round'] == circuit)]
        X_test = test.drop(['season', 'round', 'podium', 'driver_points_from_race', 'constructor_points_from_race'], axis=1)
        y_test = test.podium
        
        #scaling
        X_test = pd.DataFrame(scaler.transform(X_test), columns = X_test.columns)
        
        # make predictions
        prediction_df = pd.DataFrame(model.predict_proba(X_test), columns=['proba_0', 'proba_1'])
        prediction_df['actual'] = y_test.reset_index(drop=True)
        prediction_df['predicted'] = prediction_df.proba_1.map(lambda x: 1 if float(x) == float(prediction_df.proba_1.max()) else 0)
        

        predictions += list(prediction_df.predicted.values)

    return predictions

In [34]:
# (75, 25, 50, 10), relu, lbfgs, 0.000207

In [35]:
hidden_layer_sizes= [(80,20,40,5), (75, 25, 50, 10)][1]
activation='identity'
solver='lbfgs'
alpha=0.01623776739188721

model = MLPClassifier(
    hidden_layer_sizes=hidden_layer_sizes,
    activation=activation, 
    solver=solver, alpha=alpha, 
    random_state=1)

model.fit(X_train, y_train)

model_score = score_classification(model)
predictions = get_predictions(model)

print(model_score)

0.5


In [36]:
test = merged.query('season ==@N')
test['predicted'] = predictions
test = test[['season', 'round', 'circuit_id', 'driver', 'starting_grid' ,'podium', 'predicted', 'driver_points']]

In [37]:
test.query('predicted == 1')

,season,round,circuit_id,driver,starting_grid,podium,predicted,driver_points
2761,2021,1,bahrain,max_verstappen,1,2,1,0
2781,2021,2,imola,lewis_hamilton,1,2,1,25
2801,2021,3,portimao,valtteri_bottas,1,3,1,16
2819,2021,4,catalunya,lewis_hamilton,1,1,1,69
2839,2021,5,monaco,max_verstappen,2,1,1,80
2871,2021,6,baku,lewis_hamilton,2,15,1,101
2876,2021,7,ricard,max_verstappen,1,1,1,105
2894,2021,8,red_bull_ring,max_verstappen,1,1,1,131
2914,2021,9,red_bull_ring,max_verstappen,1,1,1,156
2953,2021,10,silverstone,max_verstappen,1,20,1,182
